# 大模型

## 模型支持

### OpenAI

In [4]:
from illufly.llm import ChatOpenAI
ChatOpenAI()("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

小兔兔爱做梦
梦见自己在云端
跳来跳去好开心
幸福笑容挂脸蛋



'小兔兔爱做梦\n梦见自己在云端\n跳来跳去好开心\n幸福笑容挂脸蛋'

In [6]:
from illufly.llm import ChatOpenAI
from illufly.io import alog
a = ChatOpenAI(model="gpt-3.5-turbo")
await a(
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    verbose=True,
    handler=alog
)

可爱小兔兔，闭上双眼做梦，
梦中有甜甜的胡萝卜和新鲜的青菜，
它在梦里飞翔，跳跃又奔跑，
醒来依然甜蜜，真是个幸福的好兔兔。



'可爱小兔兔，闭上双眼做梦，\n梦中有甜甜的胡萝卜和新鲜的青菜，\n它在梦里飞翔，跳跃又奔跑，\n醒来依然甜蜜，真是个幸福的好兔兔。'

In [7]:
a.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '可爱小兔兔，闭上双眼做梦，\n梦中有甜甜的胡萝卜和新鲜的青菜，\n它在梦里飞翔，跳跃又奔跑，\n醒来依然甜蜜，真是个幸福的好兔兔。'}]

In [8]:
await ChatOpenAI()(
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    handler=alog,
    model="gpt-3.5-turbo",  # 在调用时指定模型名称
    temperature=0.1
)

小兔子做梦真美妙，
蹦蹦跳跳在绿草地上飞翔。
梦里有彩虹和糖果山，
快乐的兔子笑开颜。



'小兔子做梦真美妙，\n蹦蹦跳跳在绿草地上飞翔。\n梦里有彩虹和糖果山，\n快乐的兔子笑开颜。'

**在调用时指定工具，返回应当调用的工具信息**

In [2]:
from illufly.llm import ChatOpenAI
from illufly.tools import ToolAgent

def get_current_weather(location: str):
    """获取城市的天气情况"""
    yield f"{location}今天是晴天。 "
    
a = ChatOpenAI(tools=[ToolAgent(get_current_weather)])
a("今天广州出门要准备什么", new_chat=True)
a.memory

Guangzhou今天是晴天。 

今天广州是晴天，出门可以准备轻便的衣物和太阳镜，带上防晒霜也是个不错的选择。祝您有一个愉快的一天！如有其他需求或问题，请随时告诉我。



[{'role': 'user', 'content': '今天广州出门要准备什么'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool', 'content': 'Guangzhou今天是晴天。 '},
 {'role': 'assistant',
  'content': '今天广州是晴天，出门可以准备轻便的衣物和太阳镜，带上防晒霜也是个不错的选择。祝您有一个愉快的一天！如有其他需求或问题，请随时告诉我。'}]

### 智谱AI

In [11]:
from illufly.llm import ChatZhipu

zhipu = ChatZhipu()
zhipu(
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    model="glm-4-flash"
)
zhipu.memory

小兔子，梦见花，
草地上，跳啊跳。
星星眨，月亮笑，
梦里花园真美妙。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '小兔子，梦见花，\n草地上，跳啊跳。\n星星眨，月亮笑，\n梦里花园真美妙。'}]

In [13]:
from illufly.io import alog
await zhipu("能改为6句？", handler=alog)
zhipu.memory

小兔子，梦见花丛开，
夜幕降，星星在窗台。
梦里跑，草地上欢跃，
彩虹桥，跨过小溪水。
月亮轻，轻轻问晚安，
兔子笑，梦中花儿香。



[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '小兔子，梦见花，\n草地上，跳啊跳。\n星星眨，月亮笑，\n梦里花园真美妙。'},
 {'role': 'user', 'content': '能改为6句？'},
 {'role': 'assistant',
  'content': '小兔子，梦儿香，\n草丛中，小眼睛亮。\n梦见花，梦里香，\n小河弯，兔子跳。\n星星闪，月亮弯，\n梦里花园，笑声连。'},
 {'role': 'user', 'content': '能改为6句？'},
 {'role': 'assistant',
  'content': '小兔子，梦见花丛开，\n夜幕降，星星在窗台。\n梦里跑，草地上欢跃，\n彩虹桥，跨过小溪水。\n月亮轻，轻轻问晚安，\n兔子笑，梦中花儿香。'}]

In [1]:
from illufly.tools import ToolAgent
from illufly.llm import ChatZhipu
from illufly.io import alog

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

zhipu = ChatZhipu(
    tools=[ToolAgent(get_current_weather)],
    model="glm-4-air"
)
await zhipu("今天广州天气如何啊", handler=alog)

call_args {'stream': True, 'model': 'glm-4-air', 'messages': [{'role': 'user', 'content': '今天广州天气如何啊'}], 'tools': [{'type': 'function', 'function': {'name': 'get_current_weather', 'description': '获取城市的天气情况', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}}, 'required': ['location']}}}]}
广州今天是晴天。 call_args {'stream': True, 'model': 'glm-4-air', 'messages': [{'role': 'user', 'content': '今天广州天气如何啊'}, {'role': 'assistant', 'content': '', 'tool_calls': [{'index': 0, 'id': 'call_202409190825157cb7df4cf5724820', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"location": "广州"}'}}]}, {'tool_call_id': 'call_202409190825157cb7df4cf5724820', 'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '}], 'tools': [{'type': 'function', 'function': {'name': 'get_current_weather', 'description': '获取城市的天气情况', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': ''}}, 'req

'今天广州的天气是晴天哦！阳光明媚，适合外出活动。'

### 通义千问

In [14]:
from illufly.llm import ChatQwen
from illufly.io import log

qwen = ChatQwen()
log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

当然可以，以下是一首关于兔子做梦的儿歌：
兔子在梦中游荡，
梦见自己变成了一只小鸟，
飞上天空自由翱翔，
醒来时发现这是个美丽的梦。



'当然可以，以下是一首关于兔子做梦的儿歌：\n兔子在梦中游荡，\n梦见自己变成了一只小鸟，\n飞上天空自由翱翔，\n醒来时发现这是个美丽的梦。'

In [15]:
qwen.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以，以下是一首关于兔子做梦的儿歌：\n兔子在梦中游荡，\n梦见自己变成了一只小鸟，\n飞上天空自由翱翔，\n醒来时发现这是个美丽的梦。'}]

In [16]:
qwen.monitor_executors()

{'CHAT_OPENAI': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'DEFAULT': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0},
 'CHAT_ZHIPU': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'CHAT_QWEN': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0}}

In [18]:
from illufly.tools import ToolAgent
from illufly.llm import ChatQwen
from illufly.io import log
import json

def get_current_weather(location: str=None):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

In [19]:
log(ChatQwen(), "今天广州天气如何啊", tools=[ToolAgent(get_current_weather)])
# log(ChatQwen(), "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

广州今天是晴天。 

广州今天的天气是晴朗的。



'广州今天的天气是晴朗的。'

## Fake

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [20]:
from illufly.llm import FakeLLM
from illufly.io import log

log(
    FakeLLM(),
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

这是一个模拟调用!



'这是一个模拟调用!'

In [21]:
FakeLLM.monitor_executors()

{'CHAT_OPENAI': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'DEFAULT': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0},
 'CHAT_ZHIPU': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'CHAT_QWEN': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0},
 'FAKE_LLM': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0}}